# Summary of best terms

- load the moderation rated tweets from database
- combine similar search terms (containing two more common words)
- show statistic sorted by highest count and delta to last week


In [41]:
from util.sql_switch import get_query_native

count_query = "select count(ds.title), ds.title, ds.created_at \
from delab_moderationrating mr \
    join delab_moderationcandidate2 d on mr.mod_candidate_id = d.id \
    join delab_tweet dt on dt.id = d.tweet_id \
    join delab_simplerequest ds on dt.simple_request_id = ds.id \
where mr.u_moderating_part is not null \
  and mr.u_moderating_part <> '' \
group by \
    ds.title, ds.created_at \
order by count(ds.title) desc"

df_term_counts = get_query_native(count_query)
# print(df_term_counts.head(8))

# squashing similar titles
import functools as ft
list_of_similar_terms = [["Thema", "bleiben"],["calm", "down"], ["constructive", "criticism"], ["explain"], ["stay", "topic"], ["whataboutism"], ["valid", "point"], ["Perspektive"], ["agree"]]

def title_group_mapper(text):
    group_title = text
    for term_group in list_of_similar_terms:
        for term in term_group:
            if term in text and group_title == text:
                group_title = ft.reduce(lambda x,y: x + "_" + y, term_group)
    return group_title

grouped_titles = df_term_counts["title"].map(title_group_mapper)
df_term_counts["grouped_titles"] = grouped_titles
df_term_counts.drop(["title", "created_at"], axis=1, inplace=True)
df_term_counts = df_term_counts.groupby("grouped_titles").sum()
df_term_counts.sort_values(by=["count"], axis=0, inplace=True, ascending=False)
df_term_counts.head(10)

using postgres
   count                     title                       created_at
0     50          "hear your view" 2022-11-17 10:50:52.533139+00:00
1     42       "can you elaborate" 2022-11-17 09:29:14.609638+00:00
2     39         "näher erläutern" 2022-11-17 07:39:36.889795+00:00
3     26        "bleib beim Thema" 2022-11-15 09:13:42.645863+00:00
4     23           "explain to me" 2022-11-15 14:31:03.881262+00:00
5     22  bitte beim Thema bleiben 2022-11-11 08:46:09.772261+00:00
6     21             "guter Punkt" 2022-11-14 13:29:39.444238+00:00
7     19              whataboutism 2022-07-12 09:58:54.675092+00:00


/home/dehne/PycharmProjects/delab/venv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
grouped_titles,
Thema_bleiben,81
"""hear your view""",50
explain,45
"""can you elaborate""",42
valid_point,41
"""näher erläutern""",39
constructive_criticism,34
stay_topic,33
agree,30


# Summary of inter-coder conflicts

- load moderation rating from database or use previous
- compute inter-coder quality
- show a list of top 10 conflicting ratings